# Rocket launches

<h1>Table of contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#API-data" data-toc-modified-id="API-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>API data</a></span><ul class="toc-item"><li><span><a href="#API-information" data-toc-modified-id="API-information-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>API information</a></span></li><li><span><a href="#Function-to-get-the-information-of-the-launches-depending-on-their-status" data-toc-modified-id="Function-to-get-the-information-of-the-launches-depending-on-their-status-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Function to get the information of the launches depending on their status</a></span></li><li><span><a href="#Execution" data-toc-modified-id="Execution-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Execution</a></span><ul class="toc-item"><li><span><a href="#Get-the-information-of-the-future-launches-(confirmed-and-the-ones-to-be-defined)" data-toc-modified-id="Get-the-information-of-the-future-launches-(confirmed-and-the-ones-to-be-defined)-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Get the information of the future launches (confirmed and the ones to be defined)</a></span></li><li><span><a href="#Merge-the-DataFrames-into-a-single-one" data-toc-modified-id="Merge-the-DataFrames-into-a-single-one-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Merge the DataFrames into a single one</a></span></li><li><span><a href="#Delete-the-columns-which-do-not-contain-relevant-information" data-toc-modified-id="Delete-the-columns-which-do-not-contain-relevant-information-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>Delete the columns which do not contain relevant information</a></span></li></ul></li><li><span><a href="#Export-to-a-CSV-file" data-toc-modified-id="Export-to-a-CSV-file-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Export to a CSV file</a></span></li></ul></li><li><span><a href="#Webscraping-data" data-toc-modified-id="Webscraping-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Webscraping data</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Input:-List-with-the-URLs-to-scrap" data-toc-modified-id="Input:-List-with-the-URLs-to-scrap-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Input: List with the URLs to scrap</a></span></li></ul></li><li><span><a href="#Define-a-spider" data-toc-modified-id="Define-a-spider-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Define a spider</a></span></li><li><span><a href="#Define-the-parser" data-toc-modified-id="Define-the-parser-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Define the parser</a></span></li><li><span><a href="#Execution" data-toc-modified-id="Execution-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Execution</a></span><ul class="toc-item"><li><span><a href="#Check-the-length-and-one-of-the-elements" data-toc-modified-id="Check-the-length-and-one-of-the-elements-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Check the length and one of the elements</a></span></li></ul></li><li><span><a href="#Convert-to-a-DataFrame" data-toc-modified-id="Convert-to-a-DataFrame-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Convert to a DataFrame</a></span></li><li><span><a href="#Export-to-CSV" data-toc-modified-id="Export-to-CSV-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Export to CSV</a></span></li></ul></li><li><span><a href="#Data-merge" data-toc-modified-id="Data-merge-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data merge</a></span><ul class="toc-item"><li><span><a href="#Import-both-csv-files" data-toc-modified-id="Import-both-csv-files-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Import both csv files</a></span><ul class="toc-item"><li><span><a href="#Export-to-a-CSV" data-toc-modified-id="Export-to-a-CSV-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Export to a CSV</a></span></li></ul></li></ul></li></ul></div>

## Imports

Libraries to import for the API and webscraping

In [1]:
import requests, random, time
from datetime import datetime, date, time
from bs4 import BeautifulSoup
import json
import pandas as pd

## API data

### API information

**mode**

The options are:
- list
- detailed

As I am interested in the list of launches and no any other information, the mode will be list

**Status**

It refers to the launch status. The options are:
- 1 = GO
- 2 = TBD
- 3 = Success
- 4 = Failure
- 5 = Hold
- 6 = In-flight
- 7 = Partial failure

### Function to get the information of the launches depending on their status

In [2]:
def get_launches(limit=100, status=2):
    
    # Initial offset of the query
    offset = 0
    # API URL
    url_api = 'https://spacelaunchnow.me/api/3.3.0/launch/?'
    
    # Loop until all the elements are requested
    while True:
        
        # Query
        q = {'mode':'list', 'limit':limit, 'offset':offset, 'status':status}
        # Request
        r = requests.get(url_api, params=q).json()
        
        # First request:
        if offset == 0:
            # Export the information to a DataFrame
            launches = pd.json_normalize(r['results'])
        else:
            #merge to previous dataframe
            df = pd.json_normalize(r['results'])
            launches = pd.concat([launches, df], ignore_index=True)
            
        # Check if all the elements have been requested and if so, break the loop
        if (offset+limit) < r['count']:
            offset += limit
        else:
            break
    
    return launches

### Execution

#### Get the information of the future launches (confirmed and the ones to be defined)

In [3]:
launches_go = get_launches(status=1)
launches_tbd = get_launches(status=2)

#### Merge the DataFrames into a single one

In [4]:
launches_info = pd.concat([launches_go, launches_tbd], ignore_index=True)

In [5]:
display(launches_info.head())

,id,url,launch_library_id,slug,name,net,window_end,window_start,mission,mission_type,pad,location,landing,landing_success,launcher,orbit,image,status.id,status.name
0,24b3b696-9253-4088-92f2-c7762f4e0a4c,http://spacelaunchnow.me/api/3.3.0/launch/24b3...,2088.0,https://spacelaunchnow.me/launch/falcon-9-bloc...,Falcon 9 Block 5 | Starlink 10,2020-08-18T14:31:00Z,2020-08-18T14:31:00Z,2020-08-18T14:31:00Z,Starlink 10,Communications,Space Launch Complex 40,"Cape Canaveral, FL, USA",OCISLY,0.0,B1049,LEO,https://spacelaunchnow-prod-east.nyc3.cdn.digi...,1,Go
1,6cdf9823-7574-4c0d-a3b2-956c7b364756,http://spacelaunchnow.me/api/3.3.0/launch/6cdf...,1973.0,https://spacelaunchnow.me/launch/delta-iv-heav...,Delta IV Heavy | NROL-44,2020-08-26T05:50:00Z,2020-08-26T10:25:00Z,2020-08-26T05:50:00Z,None,NaN,Space Launch Complex 37B,"Cape Canaveral, FL, USA",None,NaN,None,None,https://spacelaunchnow-prod-east.nyc3.cdn.digi...,1,Go
2,120c53d4-d3a1-42fe-b2a5-a63e3b11d72b,http://spacelaunchnow.me/api/3.3.0/launch/120c...,1640.0,https://spacelaunchnow.me/launch/electron-stp-...,Electron | STP-27RM,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,STP-27RM,Government/Top Secret,Rocket Lab Launch Complex 2,"Wallops Island, Virginia, USA",None,NaN,None,None,https://spacelaunchnow-prod-east.nyc3.cdn.digi...,2,TBD
3,504ba71a-bf09-48f1-9346-48c3d79e580b,http://spacelaunchnow.me/api/3.3.0/launch/504b...,NaN,https://spacelaunchnow.me/launch/momo-flight-7,MOMO | Flight 7,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,Flight 7,Suborbital,"Taiki-cho, Hokkaido, Japan","Taiki-cho, Hokkaido, Japan",None,NaN,None,Sub Orbital,https://spacelaunchnow-prod-east.nyc3.cdn.digi...,2,TBD
4,98a5da32-4719-495b-9713-fcb5c058f31c,http://spacelaunchnow.me/api/3.3.0/launch/98a5...,1359.0,https://spacelaunchnow.me/launch/falcon-9-bloc...,Falcon 9 Block 5 | SAOCOM 1B,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,SAOCOM 1B,Communications,Space Launch Complex 40,"Cape Canaveral, FL, USA",None,NaN,None,None,https://spacelaunchnow-prod-east.nyc3.cdn.digi...,2,TBD


#### Delete the columns which do not contain relevant information

In [6]:
display(launches_info.columns)

Index(['id', 'url', 'launch_library_id', 'slug', 'name', 'net', 'window_end',
       'window_start', 'mission', 'mission_type', 'pad', 'location', 'landing',
       'landing_success', 'launcher', 'orbit', 'image', 'status.id',
       'status.name'],
      dtype='object')

In [7]:
launches_info.drop(columns=['id', 'url', 'launch_library_id', 'landing',
                            'landing_success', 'launcher', 'image', 'status.id'], inplace=True)

### Export to a CSV file

In [8]:
launches_info.to_csv('output/api_data.csv', index=False)

## Webscraping data

From the API, there is a URL associated a each launch. Scrapping it, I will obtain more information about the launch and this information will be added to the DataFrame to complete it.

#### Input: List with the URLs to scrap

In [9]:
urls_list = list(launches_info.sort_values(['net'])['slug'])

### Define a spider

In [10]:
class LaunchSpider:
    """
    Parameters:
    - url: List of urls to scrape
    - sleep_interval: The time interval in seconds to delay between requests. If <0, requests will not be delayed.
    - content_parser: The function reference that will extract the intended info from the scraped content
    - output_list: it will be the final output list with the information scrapped
    """
    def __init__(self, url_list, sleep_interval=-1, content_parser=None):
        self.url_list = url_list
        self.sleep_interval = sleep_interval
        self.content_parser = content_parser
        self.output_list = []
        
    """
    Generate a random user-agent for the headers
    """
    def get_random_ua(self):
         browsers = ['Mozilla/5.0 CK={} (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
           'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148	',
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36']
        
         self.user_agent = random.choice(browsers)
    
    """
    Scrape the content of a single url
    """
    def scrape_url(self, url):
        
        # Get a random user-agent
        ua = self.get_random_ua()
        
        # Generate the headers
        headers = {'user-agent':ua}
    
        # If there is an error, print it
        try:
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code >= 400 and response.status_code < 500:
                print('The request failed because the resource either does not exist or is forbidden')
            elif response.status_code >= 300 and response.status_code < 400:
                print('Redirection error')
            elif response.status_code >= 500:
                print('Server error')
        except requests.exceptions.Timeout:
            # timeout error
            print('There has been a timeout error')
        except requests.exceptions.TooManyRedirects:
            # Too many redirects error
            print('Too many redirects')
        except requests.exceptions.SSLError:
            # SSL error
            print('SSL error')
        except requests.exceptions.RequestException as e:
            # Other unknown error
            print(f'{e}')
        
        result = self.content_parser(response.content)
        self.output_results(result)
    
    """
    Export the scraped content to the output list attribute
    """
    def output_results(self, r):
        
        # append to the output list
        self.output_list.append(r)
        
    """
    After the class is initiated, call this function to start the scraping of the urls list.
    This function uses a FOR loop to call `scrape_url()` for each url to scrape
    """
    def kickstart(self):

        for url in self.url_list:
            self.scrape_url(url)
            if self.sleep_interval > 0:
                time.sleep(self.sleep_interval)

        # give the results list as an output
        return self.output_list

### Define the parser

From each URL, we would like to obtain the description of the launch. It usually includes a description of the payload inside the rocket.

In [28]:
def launch_parser(content):

    soup = BeautifulSoup(content, 'html.parser')
    title = soup.find('h1', attrs={'class':'title'}).text
    desc = 'no information'
    date = soup.find('h3', attrs={'id':'date'}).text
    
    # Split the date to keep only year, month and day
    date = date.split()
    month = date[0]
    month = month[:3]
    day = date[1]
    year = date[2]
    date = ' '.join([month, day, year])
    
    #Transform the date into a datetime
    date = datetime.strptime(date, '%b %d, %Y,')
    
    # Some of the launches do not have any description yet and in this case there is
    # no additional information to add
    
    try: 
        desc = soup.find('div', attrs={'class':'col-md-12 mx-auto'}).find('p').text       
        return [title, desc, date]
    
    except:
        return [title, desc, date]

### Execution

In [29]:
spider = LaunchSpider(urls_list, sleep_interval=0, content_parser=launch_parser)

In [30]:
launch_list = spider.kickstart()

#### Check the length and one of the elements

In [31]:
len(launch_list)

201

In [32]:
launch_list[16]

['Atlas V 551 | STP-3', 'no information', datetime.datetime(2020, 10, 1, 0, 0)]

### Convert to a DataFrame

In [33]:
launch_description = pd.DataFrame(launch_list, columns=['name', 'description', 'date'])

In [34]:
launch_description.head()

,name,description,date
0,Falcon 9 Block 5 | Starlink 10,A batch of 58 satellites for Starlink mega-con...,2020-08-18
1,Delta IV Heavy | NROL-44,no information,2020-08-26
2,Electron | STP-27RM,A U.S. Air Force experimental spacecraft.,2020-08-31
3,MOMO | Flight 7,The sixth (Flight 6 has not happened yet) flig...,2020-08-31
4,Falcon 9 Block 5 | SAOCOM 1B,The SAOCOM 1B spacecraft is the second of the ...,2020-08-31


### Export to CSV

In [37]:
launch_description.to_csv('output/web_data.csv', index=False)

## Data merge

### Import both csv files

In [80]:
df1 = pd.read_csv('output/api_data.csv')
df2 = pd.read_csv('output/web_data.csv')

In [81]:
df1.columns

Index(['slug', 'name', 'net', 'window_end', 'window_start', 'mission',
       'mission_type', 'pad', 'location', 'orbit', 'status.name'],
      dtype='object')

In [82]:
df2.columns

Index(['name', 'description', 'date'], dtype='object')

In [83]:
df1

,slug,name,net,window_end,window_start,mission,mission_type,pad,location,orbit,status.name
0,https://spacelaunchnow.me/launch/falcon-9-bloc...,Falcon 9 Block 5 | Starlink 10,2020-08-18T14:31:00Z,2020-08-18T14:31:00Z,2020-08-18T14:31:00Z,Starlink 10,Communications,Space Launch Complex 40,"Cape Canaveral, FL, USA",LEO,Go
1,https://spacelaunchnow.me/launch/delta-iv-heav...,Delta IV Heavy | NROL-44,2020-08-26T05:50:00Z,2020-08-26T10:25:00Z,2020-08-26T05:50:00Z,NaN,NaN,Space Launch Complex 37B,"Cape Canaveral, FL, USA",NaN,Go
2,https://spacelaunchnow.me/launch/electron-stp-...,Electron | STP-27RM,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,STP-27RM,Government/Top Secret,Rocket Lab Launch Complex 2,"Wallops Island, Virginia, USA",NaN,TBD
3,https://spacelaunchnow.me/launch/momo-flight-7,MOMO | Flight 7,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,Flight 7,Suborbital,"Taiki-cho, Hokkaido, Japan","Taiki-cho, Hokkaido, Japan",Sub Orbital,TBD
4,https://spacelaunchnow.me/launch/falcon-9-bloc...,Falcon 9 Block 5 | SAOCOM 1B,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,2020-08-31T00:00:00Z,SAOCOM 1B,Communications,Space Launch Complex 40,"Cape Canaveral, FL, USA",NaN,TBD
...,...,...,...,...,...,...,...,...,...,...,...
196,https://spacelaunchnow.me/launch/sls-block-1-e...,SLS Block 1 | Europa Clipper,2025-01-01T00:00:00Z,2025-01-01T00:00:00Z,2025-01-01T00:00:00Z,Europa Clipper,Planetary Science,Launch Complex 39B,"Kennedy Space Center, FL, USA",Helio-N/A,TBD
197,https://spacelaunchnow.me/launch/sls-block-1b-...,SLS Block 1B | Artemis-5,2026-01-01T00:00:00Z,2026-01-01T00:00:00Z,2026-01-01T00:00:00Z,NaN,NaN,Launch Complex 39B,"Kennedy Space Center, FL, USA",NaN,TBD
198,https://spacelaunchnow.me/launch/sls-block-1b-...,SLS Block 1B | Artemis-6,2027-01-01T00:00:00Z,2027-01-01T00:00:00Z,2027-01-01T00:00:00Z,NaN,NaN,Launch Complex 39B,"Kennedy Space Center, FL, USA",NaN,TBD
199,https://spacelaunchnow.me/launch/sls-block-1b-...,SLS Block 1B | Artemis-7,2028-01-01T00:00:00Z,2028-01-01T00:00:00Z,2028-01-01T00:00:00Z,NaN,NaN,Launch Complex 39B,"Kennedy Space Center, FL, USA",NaN,TBD


Both dataframes have the column "name" in common

In [84]:
df3 = pd.merge(df1, df2, how='left', on='name')

In [85]:
df3.isnull().sum()

slug              0
name              0
net               0
window_end        0
window_start      0
mission         137
mission_type    137
pad               0
location          0
orbit           166
status.name       0
description      15
date             15
dtype: int64

#### Export to a CSV

In [86]:
df3.to_csv('output/merged_data.csv', index=False)